# Problem statement

As part of a trading team that makes investments based on [Fundamental Analysis](https://https://en.wikipedia.org/wiki/Fundamental_analysis).

Your purpose is to produce revenue growth forecasts for firms in a given industry based on financial charateristics of these firms and relevant economic data. Your forecasts are then used in a larger decision making system.

You have access to a dataset of 15,000 observations. Each observation $(X_i, y_i)$ consists of 4,000 economic and financial indicators that could presumably forecast revenue growth for the next quarter, and the corresponding revenue growth for that observation ($y_i$).

The corresponding data set is given below:

X.npy: https://drive.google.com/file/d/1SbC0xE1PPK0gL6J2yolIaQ07eoNVk2bM


y.py: https://drive.google.com/file/d/1HxnGlU_epSaiDppRCzkZAX8AJIkV0xvS


# Task
Construct a linear model to forecast revenue growth based on the data you have. You model will be evaluated based on the mean squared error between your predictions and the labels evaluated at a test set. The test set comes from the same distribution as the training set and the evaluation set.



In [ ]:
import gdown

file_id_X = '1SbC0xE1PPK0gL6J2yolIaQ07eoNVk2bM'

gdown.download(f'https://drive.google.com/uc?id={file_id_X}', 'X.npy', quiet=False)
X = np.load('X.npy')

Downloading...
From (original): https://drive.google.com/uc?id=1SbC0xE1PPK0gL6J2yolIaQ07eoNVk2bM
From (redirected): https://drive.google.com/uc?id=1SbC0xE1PPK0gL6J2yolIaQ07eoNVk2bM&confirm=t&uuid=85699aa0-38a6-4a0d-9412-07caa9f78343
To: /content/X.npy
100%|██████████| 240M/240M [00:03<00:00, 61.4MB/s]


In [ ]:
X = np.load('X.npy')
print('X shape:', X.shape)

X shape: (15000, 4000)


In [ ]:
file_id_y = '1HxnGlU_epSaiDppRCzkZAX8AJIkV0xvS'

gdown.download(f'https://drive.google.com/uc?id={file_id_y}', 'y.py', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1HxnGlU_epSaiDppRCzkZAX8AJIkV0xvS
To: /content/y.py
100%|██████████| 60.1k/60.1k [00:00<00:00, 43.7MB/s]


'y.py'

In [ ]:
y = np.load('y.py')
print('y shape:', y.shape)

y shape: (15000,)



that returns the predictions of you model for a new dataset X.


In [ ]:
import jax.numpy as jnp
import jax
import optax
import numpy as np

np.random.seed(100)


@jax.jit
def mse(y_true, y_predict):
    return jnp.mean((y_true - y_predict) ** 2)


@jax.jit
def dL(theta, X_tmp, y_tmp):
    def loss_fn(theta):
        y_predict = X_tmp @ theta
        return mse(y_tmp, y_predict)

    grad = jax.grad(loss_fn)(theta)

    return grad

def sample(X_tmp, y_tmp):
    N = len(X_tmp)
    batch_size = int(jnp.sqrt(N))
    idx = np.random.choice(N, batch_size)
    return X_tmp[idx], y_tmp[idx]

@jax.jit
def update_fn(theta, opt_state, X_tmp, y_tmp, alpha, beta, beta2):
    grad = dL(theta, X_tmp, y_tmp)
    optmizer = optax.adam(alpha, beta, beta2)
    updates, opt_state = optmizer.update(grad, opt_state)
    theta = optax.apply_updates(theta, updates)
    return theta, opt_state

def optimized_theta(X_train, y_train, X_val, y_val):
    n_model = 20 # Optimizing 20 different models at the same time
    max_iterations=50000
    theta = jnp.zeros(len(X_train[0]))
    alpha_list = [0.1, 0.05, 0.01, 1e-3, 1e-4, 1e-6]
    beta_list = [0.1, 0.2, 0.5, 0.9]
    beta2_list = [0.1, 0.2, 0.5, 0.9]
    alpha_vec = np.random.choice(alpha_list, n_model)
    beta_vec = np.random.choice(beta_list, n_model)
    beta2_vec = np.random.choice(beta2_list, n_model)
    hyperparams = dict({'alpha': alpha_vec, 'beta': beta_vec, 'beta2': beta2_vec})

    @jax.jit
    def stack(theta):
        return jnp.stack([theta] * n_model)

    optmizer = optax.adam(alpha_list[1], beta_vec[1], beta2_vec[1])
    opt_state = optmizer.init(theta)
    theta = stack(theta)
    opt_state = jax.tree.map(stack, opt_state)
    update = jax.jit(jax.vmap(update_fn, in_axes=(0, 0, None, None, 0, 0, 0)))


    for iteration in range(max_iterations+1):
        if iteration% 2000 ==0:
            print(f'Iterated {iteration} times')
        Xi, yi = sample(X_train, y_train)
        theta, opt_state = update(theta, opt_state, Xi, yi, alpha_vec, beta_vec, beta2_vec)

    # calculate all MSEs，find best theta
    mse_values = []
    for i in range(n_model):
        y_pred = X_val @ theta[i]
        mse_value = mse(y_val, y_pred)
        mse_values.append(mse_value)

    # choose Best MSE's theta
    index = jnp.argmin(jnp.array(mse_values))
    best_theta = theta[index]
    mse_values[index]
    print(f"Best model index: {index} \nBest training MSE: {mse(y_train, X_train @ best_theta)} \nBest validating MSE:{mse_values[index]}")

    return best_theta

def f(X):
    theta = optimized_theta(X_train, y_train, X_val, y_val)
    prediction = X @ theta
    np.save('theta.npy', theta)
    return prediction





data = jnp.load('./X.npy') # loading features and targets
y = jnp.load('./y.py')
n = len(data)
train_size = int(0.6 * len(data))
val_size = int(0.2 * n)
test_size = n - train_size - val_size

indices = np.random.permutation(len(data))
X_train = data[indices[:train_size]]
X_val = data[indices[train_size:train_size + val_size]]
X_test = data[indices[train_size + val_size:]]

y_train = y[indices[:train_size]]
y_val = y[indices[train_size:train_size + val_size]]
y_test = y[indices[train_size + val_size:]]
prediction = f(X_test)
print('Testing MSE: ',mse(prediction, y_test))


Iterated 0 times
Iterated 2000 times
Iterated 4000 times
Iterated 6000 times
Iterated 8000 times
Iterated 10000 times
Iterated 12000 times
Iterated 14000 times
Iterated 16000 times
Iterated 18000 times
Iterated 20000 times
Iterated 22000 times
Iterated 24000 times
Iterated 26000 times
Iterated 28000 times
Iterated 30000 times
Iterated 32000 times
Iterated 34000 times
Iterated 36000 times
Iterated 38000 times
Iterated 40000 times
Iterated 42000 times
Iterated 44000 times
Iterated 46000 times
Iterated 48000 times
Iterated 50000 times
Best model index: 3 
Best training MSE: 0.016544833779335022 
Best validating MSE:0.019632613286376
Testing MSE:  0.02027469
